In [1]:
from __future__ import division
from brian2 import *
import numpy as np
import scipy.io
import scipy.sparse
from sklearn import linear_model
import scipy.sparse.linalg

from sklearn.linear_model import Ridge, LogisticRegression
import scipy.signal
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from multiprocessing import Pool

import time

import os

os.chdir('/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/reservoir_Demo/ECG/data/')


/Users/liutianlin/anaconda3/lib/python3.6/site-packages/brian2/core/variables.py:174: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return np.issubdtype(np.bool, self.dtype)


In [10]:
def read_Dynapse_data(coreID,neuronID,Tm):
    core_id_array = []
    with open(coreID, 'r') as f:
        for line in f:
            core_id_array.append(int(line))
        core_id_array = np.array(core_id_array)
    
    neuron_id_array = []
    with open(neuronID, 'r') as f:
        for line in f:
            neuron_id_array.append(int(line))
        neuron_id_array = np.array(neuron_id_array)


    time_array = []
    with open(Tm, 'r') as f:
        for line in f:
            time_array.append(int(line))
        time_array = np.array(time_array)
    
    neuron_id_array0=neuron_id_array[core_id_array==0]
    time_array0=time_array[core_id_array==0]
    pulse_ts = time_array0[neuron_id_array0==1]

    t_min = np.min(time_array)
    t_max = np.max(time_array)

    unit_t = 10**(-6)*second
        
    neuron_id_array = (core_id_array-1)*256+neuron_id_array
    reservoir_neurons = neuron_id_array[core_id_array!=0]
    spike_times = time_array[core_id_array!=0]
        
    #plot(spike_times*unit_t, reservoir_neurons, '.')
    #show()
        
    print(reservoir_neurons.shape)
    print(spike_times.shape)
    
    return reservoir_neurons, spike_times





def compute_psc(spike_train, t, alpha = 4):
    if spike_train.size == 0:
        return 0
    else:
        distance_vec = unit_t * (spike_train - t) / second
        return np.sum(np.exp(alpha*distance_vec[distance_vec<0]))
  
  


def exp_smooth(spike_train, t_grid, alpha=2, inc = 1):
    n_spike = len(spike_train)
    n_t = len(t_grid)
    i_spike = 0
    i_t = 0
    sig = 0
    sig_list = []
    sig_t =0
    while i_t < n_t:
        if i_spike >= n_spike:
            delta_t = unit_t * (t_grid[i_t] - sig_t)/second
            sig = sig/np.exp(alpha*delta_t)
            sig_list.append(sig)
            sig_t = t_grid[i_t]
            i_t += 1
        elif spike_train[i_spike] < t_grid[i_t]:
            delta_t = unit_t * (spike_train[i_spike] - sig_t)/second
            sig = sig/np.exp(alpha*delta_t)+inc
            sig_t = spike_train[i_spike]
            i_spike += 1
        else:
            delta_t = unit_t * (t_grid[i_t] - sig_t)/second
            sig = sig/np.exp(alpha*delta_t)
            sig_list.append(sig)
            sig_t = t_grid[i_t]
            i_t += 1
    return sig_list   
  
    

def smooth_spike_trains(spike_times, reservoir_neurons, nrBin, n_neurons = 3 *256, alpha=2.5, inc=1):
    t_grid = np.linspace(spike_times[0], spike_times[-1], nrBin)
    
    activity_list = [exp_smooth(spike_times[reservoir_neurons==i], t_grid, alpha) for i in range(n_neurons) ]
    return np.array(activity_list)


	
def read_ECG(MatECG):
	rate = scipy.io.loadmat(MatECG)
	rate_key = list(rate.keys())
	ecg_key = rate_key[3]
	ecg_raw = rate[ecg_key]
	return ecg_raw


def read_target(trgtMat):
	rate = scipy.io.loadmat(trgtMat)
	rate_key = list(rate.keys())
	trgt_key = rate_key[3]
	trgt_raw = rate[trgt_key]
	return trgt_raw


def confusion_stat(confusion):
	TP = confusion[1,1]
	FN = confusion[1,0]
	FP = confusion[0,1]
	TN = confusion[0,0]
	#Accuracy 
	ACC = (TP + TN ) / (TP + FP + TN + FN)
	# sensitivity (recall)
	SEN = TP/(TP + FN)
	# precision
	PRE = TP/(TP + FP)
	# F1 score
	F1 = (2 * TP) / (2 * TP + FP + FN)
	return ACC, SEN, PRE, F1
    
def binarize(y, thres):
	y_b = y
	y_b[y >= thres] = 1
	y_b[y < thres] = 0
	return y_b


def scan(out, nrBins):

    width = int(len(out)/nrBins)

    result = np.zeros(nrBins)

    for i in np.arange(nrBins):
        out_in_bin = out[i*width: i*width + width]
        if np.count_nonzero(out_in_bin) == 0:
            result[i] = 0
        else :
            result[i] = 1
            
    return result

def get_rPeak_locs(rPeaks, scan_num):
    rPeaks_down =  scan(rPeaks, scan_num)
    rPeaks_down_diff = rPeaks_down[1:] - rPeaks_down[:-1]
    return np.hstack([np.array([0]),(rPeaks_down_diff==1)])



def better_predict(prdct, rPeaks_locs, avg_width, thr):
    count = prdct[rPeaks_locs.astype(int)==1]
    count[:] = 1
    prdct_i = prdct[rPeaks_locs.astype(int)==1]
    for i in range(avg_width-1):
        ind_vec = np.hstack([np.array((i+1)*[0]), rPeaks_locs[:-i-1]])
        length = prdct[ind_vec.astype(int)==1].shape[0]
        prdct_i[:length] += binarize(prdct[ind_vec.astype(int)==1], thr)
        count[:length] += 1
        #plt.plot(prdct_i)    
    return binarize(prdct_i/count, 0.5)

# Load target

In [3]:

inp = 'ECG_119'
trgt = 'Target119'
ecg = read_ECG(inp)

rPeaks = scipy.io.loadmat('Rpeaks_119')['Rpeaks'].flatten()[:324000]
rPeaks_train = rPeaks[:2 * 108000] 
rPeaks_test = rPeaks[2 * 108000:]


output = read_target(trgt)[:324000,0] # only first 15 min in this experiment

# framesPerSecond = 2
framesPerSecond = 9




train_out_pre = output[:2 * 108000] 
train_out  = scan(train_out_pre, int(2 * 108000 * framesPerSecond/ 360))




test_out_pre = output[2 * 108000:]
test_out  = scan(test_out_pre, int( 108000 * framesPerSecond / 360))


rPeaks_locs_train = get_rPeak_locs(rPeaks_train, int(2 * 108000 * 9 / 360))
rPeaks_locs_test = get_rPeak_locs(rPeaks_test, int(108000 * 9 / 360))






# Load training data

In [6]:
trainDataName = 'ecg_train'

coreID = trainDataName + '_core_id.txt'
neuronID = trainDataName + '_neuron_id.txt'
Tm = trainDataName + "_ts.txt"


reservoir_neurons_train, spike_times_train = read_Dynapse_data(coreID,neuronID,Tm)
unit_t = 10**(-6)*second


(3463259,)
(3463259,)


In [7]:
smoothingParameter = 2.5

def func_train(i):
    t_grid = np.linspace(spike_times_train[0], spike_times_train[-1], len(train_out))

    return exp_smooth(spike_times_train[reservoir_neurons_train ==i], t_grid, alpha= smoothingParameter)


p = Pool(20)
start_time = time.time()
result = p.map(func_train, range(768)) 
sicTrain = np.array(result)
print("--- %s seconds ---" % (time.time() - start_time))


--- 85.34584403038025 seconds ---


# Do cross-validation

In [8]:
nFold = 5

outSpanForTrainFold = int(len(train_out)/nFold)

outAllIndices = set(np.arange(len(train_out)))


trainIndicesAllFolds = []
validationIndicesAllFolds = []

for foldnr in np.arange(nFold):
     
    validationIndices= list(range(foldnr * outSpanForTrainFold, (foldnr + 1) * outSpanForTrainFold))
    trainIndices = list(outAllIndices - set(validationIndices))
    validationIndicesAllFolds.append(validationIndices)
    trainIndicesAllFolds.append(trainIndices)


In [11]:
ACC_tr_all = []
SEN_tr_all = []
PRE_tr_all = []
F1_tr_all = []
ACC_ts_all = []
SEN_ts_all = []
PRE_ts_all = []
F1_ts_all = []

avg_w = 3
ridgeParameter = 60000
binarize_thres = 0.2

for foldNr in np.arange(nFold):

    sicTrainFold = sicTrain[:,trainIndicesAllFolds[foldNr] ]
    sicValidationFold = sicTrain[:,validationIndicesAllFolds[foldNr] ]

    outTrainFold = train_out[trainIndicesAllFolds[foldNr]]
    outValidationFold = train_out[validationIndicesAllFolds[foldNr]]
    rPeaks_locsTrain = rPeaks_locs_train[trainIndicesAllFolds[foldNr]]
    rPeaks_locsValidation = rPeaks_locs_train[validationIndicesAllFolds[foldNr]]

    clf = Ridge(alpha = ridgeParameter)
    clf.fit(np.mat(sicTrainFold).T, outTrainFold)
    train_fold_prdct = clf.predict(np.mat(sicTrainFold).T )
    better_train_fold_prdct = better_predict(train_fold_prdct, rPeaks_locsTrain, avg_w, binarize_thres)
    better_outTrainFold = better_predict(outTrainFold, rPeaks_locsTrain, avg_w, binarize_thres)

    validation_fold_prdct = clf.predict(np.mat(sicValidationFold).T)
    better_validation_fold_prdct = better_predict(validation_fold_prdct, rPeaks_locsValidation, avg_w, binarize_thres)
    better_outValidationFold = better_predict(outValidationFold, rPeaks_locsValidation, avg_w, binarize_thres)

    better_cnf_matrix_train = confusion_matrix(better_outTrainFold, better_train_fold_prdct)
    better_cnf_matrix_validation = confusion_matrix(better_outValidationFold, better_validation_fold_prdct)

    # confusion stats
    ACC_tr, SEN_tr, PRE_tr, F1_tr = confusion_stat(better_cnf_matrix_train)
    ACC_ts, SEN_ts, PRE_ts, F1_ts = confusion_stat(better_cnf_matrix_validation)

#     print('Fold number', foldNr)
#     print(ACC_tr, SEN_tr, PRE_tr, F1_tr)
#     print(ACC_ts, SEN_ts, PRE_ts, F1_ts)
    
    ACC_tr_all.append(ACC_tr)
    SEN_tr_all.append(SEN_tr)
    PRE_tr_all.append(PRE_tr)
    F1_tr_all.append(F1_tr)
    ACC_ts_all.append(ACC_ts)
    SEN_ts_all.append(SEN_ts)
    PRE_ts_all.append(PRE_ts)
    F1_ts_all.append(F1_ts)

print('==========================================')
print('avg_w', avg_w)
print('binarize_thres', binarize_thres)
print('smoothing parameter', smoothingParameter)
print('ridge parameter', ridgeParameter)
print('------------------------------------------')
print('av ACC train', np.array(ACC_tr_all).mean())
print('av SEN train', np.array(SEN_tr_all).mean())
print('av PRE train', np.array(PRE_tr_all).mean())
print('av F1 train', np.array(F1_tr_all).mean())

print('av ACC validation', np.array(ACC_ts_all).mean())
print('av SEN validation', np.array(SEN_ts_all).mean())
print('av PRE validation', np.array(PRE_ts_all).mean())
print('av F1 validation', np.array(F1_ts_all).mean())    
print('==========================================')
    

avg_w 3
binarize_thres 0.2
smoothing parameter 2.5
ridge parameter 60000
------------------------------------------
av ACC train 0.9658080067285442
av SEN train 0.8450831936123759
av PRE train 0.9920020243750853
av F1 train 0.9125656208216297
av ACC validation 0.9588625096953096
av SEN validation 0.831574149221208
av PRE validation 0.9666666666666668
av F1 validation 0.8922260655458356


# Testing

In [15]:
dataName = 'ecg_test'

coreID = dataName + '_core_id.txt'
neuronID = dataName + '_neuron_id.txt'
Tm = dataName + "_ts.txt"


reservoir_neurons_test, spike_times_test = read_Dynapse_data(coreID,neuronID,Tm)


(1724664,)
(1724664,)


In [16]:
smoothingParameter_test =  smoothingParameter
ridgeParameter_test =  ridgeParameter
binarize_thres_test = binarize_thres

In [17]:
def func_test(i):
    t_grid = np.linspace(spike_times_test[0], spike_times_test[-1], len(test_out))

    return exp_smooth(spike_times_test[reservoir_neurons_test==i], t_grid, alpha= smoothingParameter_test)

from multiprocessing import Pool

p = Pool(20)
start_time = time.time()
result = p.map(func_test, range(768)) 
print("--- %s seconds ---" % (time.time() - start_time))
sicTest = np.array(result)


--- 42.68375515937805 seconds ---


In [18]:


clf = Ridge(alpha = ridgeParameter_test)
clf.fit(np.mat(sicTrain).T, train_out)
train_prdct = clf.predict(np.mat(sicTrain).T)
test_prdct = clf.predict(np.mat(sicTest).T)


better_train_prdct = better_predict(train_prdct, rPeaks_locs_train, avg_w, binarize_thres)
better_test_prdct = better_predict(test_prdct, rPeaks_locs_test, avg_w, binarize_thres)


In [19]:
better_train_out = better_predict(train_out, rPeaks_locs_train, avg_w, binarize_thres)
better_test_out = better_predict(test_out, rPeaks_locs_test, avg_w, binarize_thres)
better_cnf_matrix_train = confusion_matrix(better_train_out, better_train_prdct)
better_cnf_matrix_test = confusion_matrix(better_test_out, better_test_prdct)

# confusion stats
ACC_tr, SEN_tr, PRE_tr, F1_tr = confusion_stat(better_cnf_matrix_train)
ACC_ts, SEN_ts, PRE_ts, F1_ts = confusion_stat(better_cnf_matrix_test)




In [21]:
print('==============test========================')
print('smoothing parameter', smoothingParameter)
print('ridge parameter', ridgeParameter)
print('------------------------------------------')
print('ACC all train', ACC_tr)
print('SEN all train', SEN_tr)
print('PRE all train', PRE_tr)
print('F1  all train', F1_tr)

print('ACC test', ACC_ts)
print('SEN test', SEN_ts)
print('PRE test', PRE_ts)
print('F1  test', F1_ts)    
print('==========================================')


==============test========================
smoothing parameter 2.5
ridge parameter 60000
------------------------------------------
ACC all train 0.9650455927051672
SEN all train 0.841726618705036
PRE all train 0.9915254237288136
F1  all train 0.9105058365758755
ACC test 0.9786585365853658
SEN test 0.875
PRE test 1.0
F1  test 0.9333333333333333
